# Construction Crack Detection - Model Training

This notebook demonstrates the process of training a deep learning model for crack detection in construction images.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import albumentations as A
from pathlib import Path
from tqdm.notebook import tqdm

# Add project root to path
sys.path.append('..')
from crackdetect.models.segmentation import UNet
from crackdetect.data.dataset import CrackDataset
from crackdetect.data.augmentation import get_training_augmentation, get_validation_augmentation
from crackdetect.training.losses import combo_loss
from crackdetect.training.metrics import iou_score, dice_coefficient
from crackdetect.training.trainer import Trainer
from config.config import Config

## 1. Configuration

Let's set up the configuration for training.

In [ ]:
# Load configuration
config = Config()

# Override some parameters for the notebook
config.batch_size = 4
config.num_epochs = 10  # Use a small number for demonstration
config.models_dir = Path("../saved_models")
config.models_dir.mkdir(exist_ok=True, parents=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Data Preparation

Let's prepare the training and validation datasets.

In [ ]:
# Set paths
data_dir = Path("../data")
train_dir = data_dir / "train"
val_dir = data_dir / "val"

# Get augmentations
train_transform = get_training_augmentation(height=config.image_size[0], width=config.image_size[1])
val_transform = get_validation_augmentation(height=config.image_size[0], width=config.image_size[1])

# Create datasets
train_dataset = CrackDataset(
    image_dir=train_dir / "images",
    mask_dir=train_dir / "masks",
    image_size=config.image_size,
    transform=train_transform,
    preprocessing=True
)

val_dataset = CrackDataset(
    image_dir=val_dir / "images",
    mask_dir=val_dir / "masks",
    image_size=config.image_size,
    transform=val_transform,
    preprocessing=True
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=config.num_workers
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=config.num_workers
)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

## 3. Visualize Training Samples

Let's visualize some training samples to verify our data pipeline.

In [ ]:
# Get a batch of training data
train_batch = next(iter(train_loader))
images = train_batch['image']
masks = train_batch['mask']

# Visualize batch
fig, axes = plt.subplots(config.batch_size, 2, figsize=(10, 5*config.batch_size))
fig.tight_layout(pad=3.0)

for i in range(config.batch_size):
    # Convert from tensor to numpy
    image = images[i].permute(1, 2, 0).numpy()
    mask = masks[i, 0].numpy()
    
    # Display images
    axes[i, 0].imshow(image)
    axes[i, 0].set_title(f"Image {i+1}")
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask, cmap='gray')
    axes[i, 1].set_title(f"Mask {i+1}")
    axes[i, 1].axis('off')

plt.show()

## 4. Model Initialization

Let's initialize the UNet model.

In [ ]:
# Create model
model = UNet(in_channels=3, out_channels=1).to(device)

# Print model summary
from torchsummary import summary
summary(model, input_size=(3, config.image_size[0], config.image_size[1]))

## 5. Training Setup

Let's set up the training components.

In [ ]:
# Create optimizer
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# Create learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=5,
    verbose=True
)

# Create trainer
trainer = Trainer(
    model=model,
    device=device,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=combo_loss,
    optimizer=optimizer,
    lr_scheduler=scheduler,
    num_epochs=config.num_epochs,
    save_dir=config.models_dir,
    model_name="crack_segmentation_notebook"
)

## 6. Model Training

Let's train the model.

In [ ]:
# Train model
history = trainer.train()

## 7. Training Results

Let's visualize the training results.

In [ ]:
# Plot training history
epochs = range(1, config.num_epochs + 1)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(epochs, history['train_losses'], 'b-', label='Training Loss')
plt.plot(epochs, history['val_losses'], 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(epochs, history['val_ious'], 'g-')
plt.title('Validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(epochs, history['val_dices'], 'm-')
plt.title('Validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.grid(True)

plt.tight_layout()
plt.savefig(config.models_dir / "training_history.png", dpi=300)
plt.show()

## 8. Prediction Visualization

Let's visualize some predictions from the trained model.

In [ ]:
# Load the best model
best_model_path = config.models_dir / "crack_segmentation_notebook_best.pth"
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

# Get a batch of validation data
val_batch = next(iter(val_loader))
images = val_batch['image'].to(device)
masks = val_batch['mask']

# Make predictions
with torch.no_grad():
    outputs = model(images)
    preds = (outputs > 0.5).float()

# Visualize predictions
fig, axes = plt.subplots(config.batch_size, 3, figsize=(15, 5*config.batch_size))
fig.tight_layout(pad=3.0)

for i in range(config.batch_size):
    # Convert from tensor to numpy
    image = images[i].cpu().permute(1, 2, 0).numpy()
    mask = masks[i, 0].numpy()
    pred = preds[i, 0].cpu().numpy()
    
    # Calculate IoU for this prediction
    pred_tensor = torch.tensor(pred).unsqueeze(0).unsqueeze(0)
    mask_tensor = torch.tensor(mask).unsqueeze(0).unsqueeze(0)
    iou = iou_score(pred_tensor, mask_tensor).item()
    dice = dice_coefficient(pred_tensor, mask_tensor).item()
    
    # Display images
    axes[i, 0].imshow(image)
    axes[i, 0].set_title(f"Image {i+1}")
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask, cmap='gray')
    axes[i, 1].set_title(f"True Mask")
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(pred, cmap='gray')
    axes[i, 2].set_title(f"Prediction (IoU: {iou:.4f}, Dice: {dice:.4f})")
    axes[i, 2].axis('off')

plt.show()

## 9. Crack Analysis

Let's analyze the detected cracks using our CrackAnalyzer tool.

In [ ]:
from crackdetect.utils.crack_analysis import CrackAnalyzer
from crackdetect.inference.visualization import create_result_figure

# Initialize crack analyzer
analyzer = CrackAnalyzer(pixel_mm_ratio=0.1)  # Adjust the pixel_mm_ratio as needed

# Select an image from the validation set
sample_idx = 0
image = images[sample_idx].cpu().permute(1, 2, 0).numpy()
pred = preds[sample_idx, 0].cpu().numpy()

# Analyze cracks
crack_properties = analyzer.analyze_mask(pred, min_area=100)

# Visualize results
result_image = analyzer.visualize_analysis(image, pred, crack_properties)

# Display results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.tight_layout(pad=3.0)

axes[0].imshow(image)
axes[0].set_title("Image")
axes[0].axis('off')

axes[1].imshow(pred, cmap='gray')
axes[1].set_title("Predicted Mask")
axes[1].axis('off')

axes[2].imshow(result_image)
axes[2].set_title("Crack Analysis")
axes[2].axis('off')

plt.show()

# Print crack properties
print(f"Number of cracks detected: {len(crack_properties)}")
for i, props in enumerate(crack_properties):
    print(f"\nCrack #{i+1}:")
    print(f"  Severity: {props.severity}")
    print(f"  Average Width: {props.width_avg:.2f} mm")
    print(f"  Maximum Width: {props.width_max:.2f} mm")
    print(f"  Length: {props.length:.2f} mm")
    print(f"  Area: {props.area:.2f} mm²")
    print(f"  Orientation: {props.orientation:.1f}°")

## 10. Model Export

Finally, let's export the trained model for inference.

In [ ]:
# Save the model in production format
final_model_path = config.models_dir / "crack_detection_final.pth"
torch.save(model.state_dict(), final_model_path)
print(f"Model saved to {final_model_path}")

# Print instructions for using the model
print("\nTo use the model for prediction, run:")
print(f"python scripts/predict.py --image path/to/image.jpg --model {final_model_path} --output results")